In [1]:
cd ~/bench

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [2]:
import polars as pl

In [10]:
df_full = pl.read_parquet("results/overall/wordnet_general_first_full-tables.parquet")
df_dep = pl.read_parquet("results/overall/wordnet_general_first.parquet")

In [11]:
df_full.group_by(["base_table", "estimator"]).agg(
    pl.col("r2score").mean()
).sort("base_table", "estimator")

base_table,estimator,r2score
str,str,f64
"""company_employ…","""best_single_jo…",0.725684
"""company_employ…","""full_join""",0.725813
"""company_employ…","""highest_contai…",0.728485
"""company_employ…","""nojoin""",0.708723
"""company_employ…","""stepwise_greed…",0.730441
"""housing_prices…","""best_single_jo…",0.687932
"""housing_prices…","""full_join""",0.684671
"""housing_prices…","""highest_contai…",0.688642
"""housing_prices…","""nojoin""",0.679878


In [20]:
df_full = df_full.with_columns(pl.lit("full").alias("tab_version"), pl.col("base_table").str.split("-").list.first().alias("case"))
df_dep = df_dep.with_columns(pl.lit("dep").alias("tab_version"), pl.col("base_table").str.split("-").list.first().alias("case"))
df = pl.concat([df_full, df_dep])

In [25]:
ddd = df.filter(pl.col("jd_method") == "exact_matching").to_pandas().pivot_table(values=["r2score"], columns=["estimator", "tab_version"], index=["case"], aggfunc="mean")

In [28]:
ddd.style.format("{:.2f}")